In [462]:
EMBEDDING_MODEL_NAME = 'deutsche-telekom/gbert-large-paraphrase-cosine'

In [463]:
### SECTION 1.1 Load PDF files

In [464]:
import fitz  # PyMuPDF
from langchain.docstore.document import Document
import docx
import openpyxl
import os

def load_documents(files):
    """
    Loads documents from PDF, DOCX, and XLSX files.

    Parameters:
    - files: A string representing a single file path or a list of strings representing multiple file paths.

    Returns:
    - A list of Document objects loaded from the provided files.

    Raises:
    - FileNotFoundError: If any of the provided file paths do not exist.
    - Exception: For any other issues encountered during file loading.
    """
    if not isinstance(files, list):
        files = [files]  # Ensure 'files' is always a list

    documents = []
    for file_path in files:
        try:
            file_extension = os.path.splitext(file_path)[1].lower()
            if file_extension == '.pdf':
                documents.extend(load_pdf(file_path))
            elif file_extension == '.docx':
                documents.extend(load_docx(file_path))
            elif file_extension == '.xlsx':
                documents.extend(load_xlsx(file_path))
            else:
                print(f"Unsupported file type: {file_extension}")
        except FileNotFoundError as e:
            print(f"File not found: {e.filename}")
            raise
        except Exception as e:
            print(f"An error occurred while loading {file_path}: {e}")
            raise

    return documents

def load_docx(file_path):
    """
    Loads text from a DOCX file.
    """
    doc = docx.Document(file_path)
    text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
    text = clean_extra_whitespace(text)
    text = group_broken_paragraphs(text)
    return [Document(page_content=text, metadata={"source": file_path})]

def load_xlsx(file_path):
    """
    Loads text from an XLSX file.
    """
    wb = openpyxl.load_workbook(file_path)
    text = ""
    for sheet in wb.sheetnames:
        ws = wb[sheet]
        for row in ws.iter_rows(values_only=True):
            text += " ".join([str(cell) for cell in row if cell is not None]) + "\n"
    text = clean_extra_whitespace(text)
    text = group_broken_paragraphs(text)
    return [Document(page_content=text, metadata={"source": file_path})]
    
def load_pdf(file_path):
    """
    Loads documents from a PDF file using PyMuPDF.

    Parameters:
    - file_path: A string representing the path to the PDF file.

    Returns:
    - A list containing a single Document object loaded from the provided PDF file.

    Raises:
    - FileNotFoundError: If the provided file path does not exist.
    - Exception: For any other issues encountered during file loading.

    The function applies post-processing steps such as cleaning extra whitespace and grouping broken paragraphs.
    """
    try:
        # Open the PDF file
        doc = fitz.open(file_path)
        text = ""
        # Extract text from each page
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text += page.get_text("text")

        # Apply post-processing steps
        text = clean_extra_whitespace(text)
        text = group_broken_paragraphs(text)

        # Create a Document object
        document = Document(
            page_content=text,
            metadata={"source": file_path}
        )
        return [document]
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        raise
    except Exception as e:
        print(f"An error occurred while loading {file_path}: {e}")
        raise

def clean_extra_whitespace(text):
    """
    Cleans extra whitespace from the provided text.

    Parameters:
    - text: A string representing the text to be cleaned.

    Returns:
    - A string with extra whitespace removed.
    """
    return ' '.join(text.split())

def group_broken_paragraphs(text):
    """
    Groups broken paragraphs in the provided text.

    Parameters:
    - text: A string representing the text to be processed.

    Returns:
    - A string with broken paragraphs grouped.
    """
    return text.replace("\n", " ").replace("\r", " ")


In [465]:
directory = "C:/Pruebas/RAG Search/demo_docu"
# Get all files in the directory
files = os.listdir(directory)
# Filter out PDF, DOCX, and XLSX files
document_files = [f"{directory}/{file}" for file in files if file.endswith(('.pdf', '.docx', '.xlsx'))]
print(document_files)

['C:/Pruebas/RAG Search/demo_docu/Ansuchen Bildungskarenz.docx', 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_2014-02.pdf', 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'C:/Pruebas/RAG Search/demo_docu/Doku-An-Abwesenheit-Corona-Krise.xlsx', 'C:/Pruebas/RAG Search/demo_docu/Papamonat_Frühkarenzurlaub_für_Väter.pdf']


In [466]:
documents = load_documents(files=document_files)

In [467]:
len(documents)

6

In [290]:
### SECTION 1.3 Experimenting with Chunk Sizes using RecursiveCharacterTextSplitter

#### Introduction
#We are exploring the effects of various chunk sizes on text segmentation using the RecursiveCharacterTextSplitter from Langchain. This experiment is designed to refine our methods for optimally dividing text.

#### Parameters Explained
#- **Chunk Size**: This parameter sets the length of each text chunk, typically measured in characters. We begin with a predetermined chunk size to monitor how the text is segmented.
#- **Chunk Overlap**: This allows for a slight overlap between consecutive chunks to prevent ideas from being split across two chunks. Initially, the overlap is set to 10% of the chunk size, but adjustments may lead to different results.

#### Purpose
#The objective of this experiment is to investigate how varying chunk size and overlap affect text division. By testing different configurations, we seek to discover a strategy that maintains the coherence of ideas while effectively segmenting the text.


In [291]:
def split_documents(
    chunk_size: int,
    knowledge_base,
    tokenizer_name= EMBEDDING_MODEL_NAME,
):
    """
    Splits documents into chunks of maximum size `chunk_size` tokens, using a specified tokenizer.

    Parameters:
    - chunk_size: The maximum number of tokens for each chunk.
    - knowledge_base: A list of LangchainDocument objects to be split.
    - tokenizer_name: (Optional) The name of the tokenizer to use. Defaults to `EMBEDDING_MODEL_NAME`.

    Returns:
    - A list of LangchainDocument objects, each representing a chunk. Duplicates are removed based on `page_content`.

    Raises:
    - ImportError: If necessary modules for tokenization are not available.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
    )

    docs_processed = (text_splitter.split_documents([doc]) for doc in knowledge_base)
    # Flatten list and remove duplicates more efficiently
    unique_texts = set()
    docs_processed_unique = []
    for doc_chunk in docs_processed:
        for doc in doc_chunk:
            if doc.page_content not in unique_texts:
                unique_texts.add(doc.page_content)
                docs_processed_unique.append(doc)

    return docs_processed_unique

In [292]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer

docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    documents,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

print(f"Number of chunks: {len(docs_processed)}")

Number of chunks: 88


In [293]:
#### SECTION 1.4 The Embedding Model

In [294]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm.autonotebook import tqdm, trange

embedding_model = HuggingFaceEmbeddings(model_name="deutsche-telekom/gbert-large-paraphrase-cosine")

In [295]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='deutsche-telekom/gbert-large-paraphrase-cosine', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [296]:
### Vector Store

In [297]:
# Ruta a la carpeta local con archivos PDF, DOCX y XLSX
#folder_path = "C:/Pruebas/RAG Search/demo_docu_3" #"C:/Users/hernandc/RAG Test/RAG Advanced/data" #"C:/Pruebas/RAG Search/Documentos" #demo_docu_2" #demo_docu" #Documentos"

# Lista para almacenar los datos de todos los documentos
data = []

In [298]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm
#from langchain.vectorstores.milvus import Milvus
from langchain_milvus import Milvus
from pymilvus import MilvusClient
import os

# Inicializar el cliente de Milvus
client = MilvusClient()

# Nombre de la colección
COLLECTION_NAME = "uni_test_5_2" #"uni_test" "rag_milvus_webinar"

# Verificar si la colección ya existe
if client.has_collection(COLLECTION_NAME):
    print(f"Cargando la colección existente: {COLLECTION_NAME}")

    # Crear una lista para almacenar los documentos procesados
else:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # the maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap=100,  # the number of characters to overlap between chunks
        add_start_index=True,  # If `True`, includes chunk's start index in metadata
        strip_whitespace=True,  # If `True`, strips whitespace from the start and end of every document
    )
    all_splits = text_splitter.split_documents(docs_processed)

    docs_processed = []
    
    # Iterar sobre los documentos y mostrar el progreso
    for doc in tqdm(all_splits, desc="Procesando documentos"):
        docs_processed.append(doc)

    # Supongamos que Milvus.from_documents permite la inserción en lotes
    batch_size = 1  # Tamaño del lote
    num_batches = len(docs_processed) // batch_size + (1 if len(docs_processed) % batch_size != 0 else 0)

    # Crear el vectorstore con los documentos procesados en lotes
    for i in tqdm(range(num_batches), desc="Insertando documentos en Milvus"):
        batch = docs_processed[i * batch_size:(i + 1) * batch_size]
        Milvus.from_documents(documents=batch, embedding=embedding_model, collection_name=COLLECTION_NAME)

vectorstore = Milvus(collection_name=COLLECTION_NAME, embedding_function=embedding_model)

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 49be3e3606f9475eaecf3110cd6e11d4
Insertando documentos en Milvus: 100%|███████████████████████████████████████████████| 168/168 [03:12<00:00,  1.15s/it]


In [299]:
retriever = vectorstore.as_retriever()

In [300]:
#similar_chunks = retriever.get_relevant_documents(query="﻿﻿Mein Vater ist gestorben, wie viel Tage Sonderurlaub bekomme ich?")
similar_chunks = retriever.invoke(input="﻿﻿Mein Vater ist gestorben, wie viel Tage Sonderurlaub bekomme ich?")
similar_chunks

[Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'start_index': 0, 'pk': 451633302904136616}, page_content='Abs. 2 Arbeitsruhegesetz [ARG]) für die gemäß ihren religiösen Vorschriften festgelegten Feiertage die unbedingt erforderliche freie Zeit unter Fortzahlung des Entgeltes im Höchstausmaß von zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr. Diese Feiertage sind vom/von der Dienstnehmer/in unverzüglich nach Abschluss des Arbeitsvertrages (bzw bei bestehenden Dienst- /Arbeitsverhältnissen innerhalb eines Monats nach'),
 Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/BV_Sonderurlaube_Dienstverhinderungen.pdf', 'start_index': 0, 'pk': 451633302904136612}, page_content='f. f. f. f. Teilnahme an der Bestattung naher Angehöriger, die nicht im gemeinsamen Haushalt gelebt haben ein Arbeitstag ein Arbeitstag ein Arbeitstag ein Ar

In [301]:
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content)

--------------------------------- chunk # 0 -------------------------------------
Abs. 2 Arbeitsruhegesetz [ARG]) für die gemäß ihren religiösen Vorschriften festgelegten Feiertage die unbedingt erforderliche freie Zeit unter Fortzahlung des Entgeltes im Höchstausmaß von zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr zwei Arbeitstagen pro Kalenderjahr. Diese Feiertage sind vom/von der Dienstnehmer/in unverzüglich nach Abschluss des Arbeitsvertrages (bzw bei bestehenden Dienst- /Arbeitsverhältnissen innerhalb eines Monats nach
--------------------------------- chunk # 1 -------------------------------------
f. f. f. f. Teilnahme an der Bestattung naher Angehöriger, die nicht im gemeinsamen Haushalt gelebt haben ein Arbeitstag ein Arbeitstag ein Arbeitstag ein Arbeitstag Seite 5 von 7 g. g. g. g. Teilnahme an der Bestattung der Eltern des Ehepartners/eingetragenen Partners/Lebensgefährten ein Arbeitstag ein Arbeitstag ein Arbeitst

In [302]:
def retrieve_context(query, retriever):
    """
    Retrieves and reranks documents relevant to a given query.

    Parameters:
    - query: The search query as a string.
    - retriever: An instance of a Retriever class used to fetch initial documents.

    Returns:
    - A list of reranked documents deemed relevant to the query.

    """
    retrieved_docs = retriever.invoke(input=query)

    return retrieved_docs

In [303]:
### SECTION 1.5 Putting Everything Together

In [304]:
### Using AZURE OPENAI API

#Moving forward, we will be using both openai LLM and Embedding model.

In [305]:
from langchain.prompts import ChatPromptTemplate
prompt_template = ChatPromptTemplate.from_template(
        (
            "Bitte beantworte die folgende Anfrage auf der Grundlage des angegebenen `Kontext`, der auf die Anfrage folgt.\n"
            "Wenn du die Antwort nicht weißt, dann sag einfach 'Ich weiß es nicht'.\n"
            "Anfrage: {question}\n"
            "Kontext: ```{context}```\n"
        )
    )

In [306]:
from dotenv import load_dotenv
import os
from pathlib import Path

dotenv_path = Path(r'C:\Users\hernandc\RAG Test\apikeys.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [307]:
from openai import AzureOpenAI

# Configurar el cliente de Azure OpenAI
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

AZURE_OPENAI_MODEL = "gpt-4-turbo" # "gpt-4"  # Reemplaza esto con el nombre de tu despliegue de GPT-4 en Azure

In [308]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage

def azure_openai_call(prompt):
    # Si el prompt es un objeto HumanMessage, extraemos su contenido
    if isinstance(prompt, HumanMessage):
        prompt_content = prompt.content
    else:
        prompt_content = str(prompt)
    
    response = client.chat.completions.create(
        model=AZURE_OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt_content}
        ]
    )
    return response.choices[0].message.content

llm = (lambda x: azure_openai_call(x))  # Envolver la llamada en una función lambda
chain = prompt_template | llm | StrOutputParser()

In [309]:
query = "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"

context = retrieve_context(
        query, retriever=retriever,
    )

In [310]:
context

[Document(metadata={'start_index': 0, 'pk': 451633302904136368, 'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf'}, page_content='um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben'),
 Document(metadata={'start_index': 417, 'pk': 451633302904136390, 'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf'}, page_content='am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für 

In [311]:
text = ""
for ch in context:
    text += ch.page_content

In [312]:
print(text)

um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Lebenam Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für Weiterbildung Krems. © Uni Graz/WildundWunderbar © Uni Graz/WildundWunderbar Organigramm FAKULTÄTEN* UNIRAT Der Unirat besteht aus 9Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Fo

In [313]:
response = chain.invoke({"context": context, "question": query})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
[inputs]
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableSequence > chain:RunnableLambda] [2.09s] Exiting Chain run with output:
{
  "output": "Auf Grundlage des Kontextes, lässt sich die Anfrage wie folgt beantworten:\n\nDer Rektor der Universität Graz ist seit Oktober 2022 Peter Riedler. Das Geburtsjahr von Peter Riedler wird im Kontext jedoch nicht angegeben.\n\nDaher lautet die vollständige Antwort: Peter Riedler ist seit Oktober 2022 der Rektor der Universität Graz. Sein Geburtsjahr ist im gegebenen Kontext nicht enthalten."
}
[chain/start] [chain:RunnableSequence > parser:StrOutputParser] Entering Parser run with inpu

In [314]:
print(response)

Auf Grundlage des Kontextes, lässt sich die Anfrage wie folgt beantworten:

Der Rektor der Universität Graz ist seit Oktober 2022 Peter Riedler. Das Geburtsjahr von Peter Riedler wird im Kontext jedoch nicht angegeben.

Daher lautet die vollständige Antwort: Peter Riedler ist seit Oktober 2022 der Rektor der Universität Graz. Sein Geburtsjahr ist im gegebenen Kontext nicht enthalten.


In [315]:
## SECTION 2.1 Reranking

#In this section we will look into different re-ranking techniques that you can use in your RAG applications

#Based on this gist: https://gist.github.com/virattt/bf13f748c6b4763b6c6215c8659c02f6

In [316]:
#from langchain.vectorstores import Chroma
from langchain_milvus import Milvus

def get_retriever(embedding_model, collection_name, top_k=10):
    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """
    # Example of parameter validation (optional)
    if top_k < 1:
        raise ValueError("top_k must be at least 1")

    try:
        vector_store = Milvus(collection_name=collection_name, embedding_function=embedding_model)

        retriever = vector_store.as_retriever(k=top_k)
        # retriever.k = top_k

        return retriever
    except Exception as e:
        print(f"An error occurred while initializing the retriever: {e}")
        raise

In [317]:
retriever_rerank = get_retriever(embedding_model, collection_name=COLLECTION_NAME, top_k=10)

In [318]:
retriever_rerank

VectorStoreRetriever(tags=['Milvus', 'HuggingFaceEmbeddings'], vectorstore=<langchain_milvus.vectorstores.milvus.Milvus object at 0x000001DA9BB8C910>)

In [319]:
query = "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
similar_chunks = retrieve_context(
        query, retriever=retriever_rerank,
    )

In [320]:
similar_chunks

[Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'start_index': 0, 'pk': 451633302904136368}, page_content='um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben'),
 Document(metadata={'source': 'C:/Pruebas/RAG Search/demo_docu/Broschuere_Int-Mitarbeitende_2023_WEB.pdf', 'start_index': 417, 'pk': 451633302904136390}, page_content='am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für 

In [321]:
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content[:800])

--------------------------------- chunk # 0 -------------------------------------
um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben
--------------------------------- chunk # 1 -------------------------------------
am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für Weiterbildung Krems. © Uni Graz/WildundWunderbar © Uni Graz/WildundWunderbar Organigramm FAKULTÄTEN* UNIRAT Der Unirat besteht aus 9
--------------------------------- chunk 

In [322]:
####  SECTION 2.1.2 Rerank using custom german Reranking Model: https://huggingface.co/deepset/gbert-base-germandpr-reranking

In [323]:
from transformers import AutoTokenizer, AutoModel

# Load the tokenizer and the model
RERANKING_MODEL_NAME = "deepset/gbert-base-germandpr-reranking"
tokenizer = AutoTokenizer.from_pretrained(RERANKING_MODEL_NAME)
model = AutoModel.from_pretrained(RERANKING_MODEL_NAME)

In [324]:
#### NOTE: This run was on CPU, GPU will be much faster.

In [325]:
import torch

start = time.time()
scores = []

# Function to compute MaxSim
def maxsim(query_embedding, document_embedding):
    # Expand dimensions for broadcasting
    # Query: [batch_size, query_length, embedding_size] -> [batch_size, query_length, 1, embedding_size]
    # Document: [batch_size, doc_length, embedding_size] -> [batch_size, 1, doc_length, embedding_size]
    expanded_query = query_embedding.unsqueeze(2)
    expanded_doc = document_embedding.unsqueeze(1)

    # Compute cosine similarity across the embedding dimension
    sim_matrix = torch.nn.functional.cosine_similarity(expanded_query, expanded_doc, dim=-1)

    # Take the maximum similarity for each query token (across all document tokens)
    # sim_matrix shape: [batch_size, query_length, doc_length]
    max_sim_scores, _ = torch.max(sim_matrix, dim=2)

    # Average these maximum scores across all query tokens
    avg_max_sim = torch.mean(max_sim_scores, dim=1)
    return avg_max_sim

# Encode the query
query_encoding = tokenizer(query, return_tensors='pt')
query_embedding = model(**query_encoding).last_hidden_state.mean(dim=1)

# Get score for each document
for document in similar_chunks:
    document_encoding = tokenizer(document.page_content, return_tensors='pt', truncation=True, max_length=512)
    document_embedding = model(**document_encoding).last_hidden_state

    # Calculate MaxSim score
    score = maxsim(query_embedding.unsqueeze(0), document_embedding)
    scores.append({
        "score": score.item(),
        "document": document.page_content,
    })

print(f"Took {time.time() - start} seconds to re-rank documents with gbert-base-germandpr-reranking.")

Took 1.1533102989196777 seconds to re-rank documents with gbert-base-germandpr-reranking.


In [326]:
import json

# Sort the scores by highest to lowest and print
sorted_data = sorted(scores, key=lambda x: x['score'], reverse=True)
print(json.dumps(sorted_data, indent=2))

[
  {
    "score": 0.6068548560142517,
    "document": "Graz. Seine beruflichen Stationen f\u00fchrten ihn unter anderem ins Europ\u00e4ische Parlament in Br\u00fcssel, in die Steierm\u00e4rki\u00ad sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 gesch\u00e4ftsf\u00fch\u00ad render Rektor der Universit\u00e4t Graz. Seit Oktober 2022 ist er Rektor. Vizerektor f\u00fcr Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007"
  },
  {
    "score": 0.5806236267089844,
    "document": "akgl.uni-graz.at Universit\u00e4tsbibliothek ub.uni-graz.at Studienabteilung +43 316 380 1162 / 2192 zulassung.international@uni-graz.at 17 Early stage researchers (PhDs & PostDocs) Der Universit\u00e4t Graz ist die Unterst\u00fctzung, Aus- und Weiterbildung ihrer \u201eearly stage researcher\u201c"
  },
  {
    "score": 0.57917118072

In [327]:
####  SECTION 2.1.3 Rerank using Cohere

In [328]:
import cohere

# Get your cohere API key on: www.cohere.com
co = cohere.Client(os.getenv("COHERE_API_KEY"))
#co = cohere.Client(userdata.get("COHERE_API_KEY"))

documents = [f"{doc.page_content}" for doc in similar_chunks]

In [329]:
print(documents)

['um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben', 'am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für Weiterbildung Krems. © Uni Graz/WildundWunderbar © Uni Graz/WildundWunderbar Organigramm FAKULTÄTEN* UNIRAT Der Unirat besteht aus 9', 'Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki\xad sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirt

In [330]:
import time

# Example query and passages
start = time.time()

results = co.rerank(query=query,
                    documents=documents,
                    top_n=4,
                    model="rerank-multilingual-v3.0", #"rerank-english-v3.0",
                    return_documents=True)
print(f"Took {time.time() - start} seconds to re-rank documents with Cohere.")

Took 0.33408427238464355 seconds to re-rank documents with Cohere.


In [331]:
results.results

[RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki\xad sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 geschäftsfüh\xad render Rektor der Universität Graz. Seit Oktober 2022 ist er Rektor. Vizerektor für Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007'), index=2, relevance_score=0.23265865),
 RerankResponseResultsItem(document=RerankResponseResultsItemDocument(text='um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche

In [332]:
for idx, r in enumerate(results.results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}, Relevance Score: {r.relevance_score:.2f}")
  print(f"Document: {r.document.text}")
  print(f"------------------------------------------------------------------------------")

Document Rank: 1, Document Index: 2, Relevance Score: 0.23
Document: Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 geschäftsfüh­ render Rektor der Universität Graz. Seit Oktober 2022 ist er Rektor. Vizerektor für Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007
------------------------------------------------------------------------------
Document Rank: 2, Document Index: 0, Relevance Score: 0.01
Document: um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des 

In [333]:
## SECTION 3 Query Expansion

#- Query Decomposition
#- Multi-query Retrieval

In [334]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """Du bist ein hilfreicher Assistent, der mehrere Unterfragen zu einer Eingangsfrage erstellt.
Das Ziel ist es, die Eingabe in eine Reihe von Unterproblemen/Unterfragen zu zerlegen, die isoliert beantwortet werden können.
Generiere mehrere Suchanfragen mit Bezug zu: {Frage}
Ausgabe (5 Abfragen):"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [335]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

In [336]:
query

'Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?'

In [337]:
# Chain
generate_queries_decomposition = ( decomposition_prompt | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
questions = generate_queries_decomposition.invoke({"Frage":query})

print(questions)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "Frage": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "Frage": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Du bist ein hilfreicher Assistent, der mehrere Unterfragen zu einer Eingangsfrage erstellt.\nDas Ziel ist es, die Eingabe in eine Reihe von Unterproblemen/Unterfragen zu zerlegen, die isoliert beantwortet werden können.\nGeneriere mehrere Suchanfragen mit Bezug zu: Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?\nAusgabe (5 Abfragen):"
  ]
}
[llm/end] [chain:RunnableSequence > llm:

In [338]:
#### multi-query retrieval

In [339]:
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [340]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema.runnable import RunnablePassthrough


final_retriever = MultiQueryRetriever.from_llm(retriever_rerank, llm)

In [341]:
print(query)

Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?


In [342]:
prompt_template = """
Du bist ein hilfreicher Assistent. Gib Antworten auf die Fragen der Benutzer auf der Grundlage des bereitgestellten Kontexts.

Kontext:
{Kontext}

Frage:
{Frage}


"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = {"Frage": RunnablePassthrough(), "Kontext": final_retriever} \
        | prompt \
        | llm \
        | StrOutputParser() \

result = chain.invoke(query)

# display(HTML(result))
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext>] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > retriever:Retriever > chain:RunnableSequence] Entering Chain run with input:
{
  "

INFO:langchain.retrievers.multi_query:Generated queries: ['Wer ist der aktuelle Rektor der Universität Graz und wann ist sein Geburtsjahr?', 'Könnten Sie mir den Namen und das Geburtsjahr des derzeitigen Rektors der Universität Graz mitteilen?', 'Wer bekleidet das Amt des Rektors an der Universität Graz und welches Geburtsdatum hat diese Person?']


[llm/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > retriever:Retriever > chain:RunnableSequence > llm:AzureChatOpenAI] [1.45s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Wer ist der aktuelle Rektor der Universität Graz und wann ist sein Geburtsjahr?\n\nKönnten Sie mir den Namen und das Geburtsjahr des derzeitigen Rektors der Universität Graz mitteilen?\n\nWer bekleidet das Amt des Rektors an der Universität Graz und welches Geburtsdatum hat diese Person?",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null,
          "content_filter_results": {}
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Wer ist der aktuelle Rektor der Universität Graz und wa

In [343]:
## SECTION-4 Query Expansion -  Hypothetical Document Embeddings (HyDE)

In [344]:
from langchain.chains import HypotheticalDocumentEmbedder

In [345]:
# https://arxiv.org/pdf/2212.10496.pdf
# embeddings = HypotheticalDocumentEmbedder.from_llm(llm, embedding_model, "web_search")

from langchain.prompts import PromptTemplate
#from langchain.embeddings import HypotheticalDocumentEmbedder
#from langchain.chains import LLMChain

CUSTOM_HYDE_PROMPT = PromptTemplate(
    input_variables=["Frage"],
    template="Bitte schreib einen Text, um die Frage zu beantworten \nFrage: {Frage}\nPassage:"
)

embeddings = HypotheticalDocumentEmbedder.from_llm(
    llm,
    embedding_model,
    "web_search",
    custom_prompt = CUSTOM_HYDE_PROMPT
)

In [346]:
query #= "What are the specific factors contributing to Airbnb's increased operational expenses in the last fiscal year?"

'Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?'

In [347]:
embeddings

HypotheticalDocumentEmbedder(base_embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='deutsche-telekom/gbert-large-paraphrase-cosine', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['Frage'], template='Bitte schreib einen Text, um die Frage zu beantworten \nFrage: {Frage}\nPassage:'), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001DA9BBB1210>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0

In [348]:
import langchain
print(embeddings.llm_chain.prompt)
langchain.debug = True

input_variables=['Frage'] template='Bitte schreib einen Text, um die Frage zu beantworten \nFrage: {Frage}\nPassage:'


In [349]:
# Now we can use it as any embedding class!
result = embeddings.embed_query(query)

[llm/start] [llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Bitte schreib einen Text, um die Frage zu beantworten \nFrage: Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?\nPassage:"
  ]
}
[llm/end] [llm:AzureChatOpenAI] [2.50s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Es tut mir leid, aber die Passage, die du angegeben hast, enthält nicht die notwendigen Informationen, um die Frage zu beantworten. Um herauszufinden, wer der Rektor der Universität Graz ist und in welchem Jahr er geboren wurde, empfehle ich, die offizielle Website der Universität Graz zu besuchen oder aktuelle Nachrichtenquellen zu konsultieren. Websites von Universitäten haben oft eine Sektion über die Leitung, wo solche Informationen veröffentlicht werden.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null,
          "content_filter_results": {}
        },
        "type": "Chat

In [350]:
# print(result)

In [351]:
hyde_retriever = get_retriever(embeddings, collection_name=COLLECTION_NAME, top_k=5)

In [352]:
query #= "What are the specific factors contributing to Airbnb's increased operational expenses in the last fiscal year?"
similar_chunks = retrieve_context(
        query, retriever=hyde_retriever,
    )

[llm/start] [llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Bitte schreib einen Text, um die Frage zu beantworten \nFrage: Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?\nPassage:"
  ]
}
[llm/end] [llm:AzureChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Der aktuelle Rektor der Universität Graz ist Peter Riedler. Geboren wurde er im Jahr 1967.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null,
          "content_filter_results": {}
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "Der aktuelle Rektor der Universität Graz ist Peter Riedler. Geboren wurde er im Jahr 1967.",
            "response_metad

In [353]:
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content[:500])

--------------------------------- chunk # 0 -------------------------------------
© Uni Graz/Schreyer 6 Die Universitätsleitung Rektor Peter Riedler Dr. Peter Riedler, 1969 in Graz geboren, promovierte im Bereich Völkerrecht und Europarecht an der Universität Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische
--------------------------------- chunk # 1 -------------------------------------
Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 geschäftsfüh­ render Rektor der Universität Graz. Seit Oktober 2022 ist er Rektor. Vizerektor für Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007
--------------------------------- chunk # 2 -------------------------------------
Au

In [354]:
langchain.debug = False

In [355]:
prompt_template = """
Du bist ein hilfreicher Assistent. Gib Antworten auf die Fragen der Benutzer auf der Grundlage des bereitgestellten Kontexts.

Kontext:
{Kontext}

Frage:
{Frage}


"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = {"Frage": RunnablePassthrough(), "Kontext": hyde_retriever} \
        | prompt \
        | llm \
        | StrOutputParser() \

result = chain.invoke(query)

# display(HTML(result))
print(result)

Der Rektor der Universität Graz ist Dr. Peter Riedler, geboren im Jahr 1969.


In [356]:
#### multiple llms

In [357]:
langchain.debug=True

In [358]:
multi_llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    n=4
)

In [359]:
embeddings = HypotheticalDocumentEmbedder.from_llm(
    multi_llm, embedding_model, "web_search"
)

result = embeddings.embed_query(query)

[llm/start] [llm:AzureChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Please write a passage to answer the question \nQuestion: Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?\nPassage:"
  ]
}
[llm/end] [llm:AzureChatOpenAI] [1.77s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Der Rektor der Universität Graz ist derzeit Martin Polaschek. Er wurde im Jahr 1965 geboren. Polaschek hat eine beeindruckende akademische Karriere hinter sich und spielt eine zentrale Rolle in der Führung und Entwicklung der Universität Graz.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null,
          "content_filter_results": {}
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kw

In [360]:
## SECTION-5 Fusion Retriever / Hybrid Search

In [372]:
from langchain_milvus import Milvus

def get_retriever(embedding_model, collection_name, top_k=3):
    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """
    # Example of parameter validation (optional)
    if top_k < 1:
        raise ValueError("top_k must be at least 1")

    try:
        vector_store = Milvus(collection_name=collection_name, embedding_function=embedding_model)

        retriever = vector_store.as_retriever(k=top_k)
        # retriever.k = top_k

        return retriever
    except Exception as e:
        print(f"An error occurred while initializing the retriever: {e}")
        raise

In [373]:

from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_milvus import Milvus


def get_ensemble_retriever(docs, embedding_model, collection_name=COLLECTION_NAME, top_k=3):
    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """

    # Hybrid search
    # Example of parameter validation (optional)
    if top_k < 1:
        raise ValueError("top_k must be at least 1")

    try:
        vector_store = Milvus(collection_name=collection_name, embedding_function=embedding_model)

        retriever = vector_store.as_retriever(search_kwargs={"k":top_k})
        # retriever.k = top_k

        # add keyword search
        keyword_retriever = BM25Retriever.from_documents(docs)
        keyword_retriever.k =  3

        ensemble_retriever = EnsembleRetriever(retrievers=[retriever,
                                                    keyword_retriever],
                                        weights=[0.5, 0.5])

        return ensemble_retriever
    except Exception as e:
        print(f"An error occurred while initializing the retriever: {e}")
        raise

In [374]:
hybrid_retriever = get_ensemble_retriever(all_splits, embedding_model, collection_name=COLLECTION_NAME, top_k=5)

In [375]:
hybrid_retriever

EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['Milvus', 'HuggingFaceEmbeddings'], vectorstore=<langchain_milvus.vectorstores.milvus.Milvus object at 0x000001DA9B7ABA60>, search_kwargs={'k': 5}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x000001DA9B7AB730>, k=3)], weights=[0.5, 0.5])

In [376]:
query #= "What are the specific factors contributing to Airbnb's increased operational expenses in the last fiscal year?"
similar_chunks = retrieve_context(
        query, retriever=hybrid_retriever,
    )

In [377]:
query #= "What are the specific factors contributing to Airbnb's increased operational expenses in the last fiscal year?"
similar_chunks = retrieve_context(
        query, retriever=hybrid_retriever,
    )

for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content[:500])

--------------------------------- chunk # 0 -------------------------------------
Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 geschäftsfüh­ render Rektor der Universität Graz. Seit Oktober 2022 ist er Rektor. Vizerektor für Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007
--------------------------------- chunk # 1 -------------------------------------
um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen

In [378]:
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content[:500])

--------------------------------- chunk # 0 -------------------------------------
Graz. Seine beruflichen Stationen führten ihn unter anderem ins Europäische Parlament in Brüssel, in die Steiermärki­ sche Landesregierung, 2002 ins Bundeskanzleramt, mit den Schwerpunkten Wirtschaft und Forschung, sowie 2007 als Director of Public Affairs zur AVL List GmbH in Graz. 2011 wurde Riedler Vizerektor, im Dezember 2021 geschäftsfüh­ render Rektor der Universität Graz. Seit Oktober 2022 ist er Rektor. Vizerektor für Forschung Univ.-Prof. Dr. Joachim Reidl Joachim Reidl ist seit 2007
--------------------------------- chunk # 1 -------------------------------------
um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen

In [379]:
prompt_template = """
Du bist ein hilfreicher Assistent. Gib Antworten auf die Fragen der Benutzer auf der Grundlage des bereitgestellten Kontexts.

Kontext:
{Kontext}

Frage:
{Frage}


"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = {"Frage": RunnablePassthrough(), "Kontext": hybrid_retriever} \
        | prompt \
        | llm \
        | StrOutputParser() \

result = chain.invoke(query)

# display(HTML(result))
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext>] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext>] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableSequence > p

In [468]:
## SECTION-6 Context Expansion - Parent Document Retriever

In [469]:
from langchain_milvus import Milvus

def get_retriever(docs, embedding_model, top_k=3):
    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """
    # Example of parameter validation (optional)
    if top_k < 1:
        raise ValueError("top_k must be at least 1")

    try:
        vector_store = Milvus(collection_name=collection_name, embedding_function=embedding_model)

        retriever = vector_store.as_retriever(k=top_k)
        # retriever.k = top_k

        return retriever
    except Exception as e:
        print(f"An error occurred while initializing the retriever: {e}")
        raise


In [470]:
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever

tokenizer_name= EMBEDDING_MODEL_NAME
chunk_size = 512

def create_parent_retriever(
    docs,
    embeddings_model,
    collection_name="split_documents",
    top_k=5,
#    persist_directory=None,
):

    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - collection_name: The name of the collection
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.
    - persist_directory: directory where you want to store your vectorDB

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """

    parent_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n\n", "\n\n", "\n", ".", ""],
        is_separator_regex=False,
    )

    # This text splitter is used to create the child documents
    child_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=int(chunk_size / 2),
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n\n", "\n\n", "\n", ".", ""],
        is_separator_regex=False,
    )

    # The vectorstore to use to index the child chunks
    vector_store = Milvus(collection_name=collection_name, embedding_function=embedding_model, auto_id=True) #, persist_directory=persist_directory)

    # The storage layer for the parent documents
    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vector_store,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        k=10,
    )
    retriever.add_documents(docs)


    return retriever, vectorstore.as_retriever()

In [ ]:
''' ESTE MÉTODO FUNCIONA Y PERMITE MOSTRAR UNA BARRA DE PROGRESO AL AGREGAR DOCUMENTOS, PERO DESPUÉS AL USARLO NO ENTREGA LOS MISMOS CHUNKS QUE CON EL MÉTODO DE ARRIBA
import logging
logging.basicConfig(level=logging.INFO)

from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from tqdm import tqdm
import logging

tokenizer_name = EMBEDDING_MODEL_NAME
chunk_size = 512

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_parent_retriever(
    docs,
    embeddings_model,
    collection_name="split_documents",
    top_k=5,
):
    """
    Initializes a retriever object to fetch the top_k most relevant documents based on cosine similarity.

    Parameters:
    - docs: A list of documents to be indexed and retrieved.
    - embedding_model: The embedding model to use for generating document embeddings.
    - collection_name: The name of the collection
    - top_k: The number of top relevant documents to retrieve. Defaults to 3.

    Returns:
    - A retriever object configured to retrieve the top_k relevant documents.

    Raises:
    - ValueError: If any input parameter is invalid.
    """

    parent_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n\n", "\n\n", "\n", ".", ""],
        is_separator_regex=False,
    )

    child_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=int(chunk_size / 2),
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=["\n\n\n", "\n\n", "\n", ".", ""],
        is_separator_regex=False,
    )

    vector_store = Milvus(collection_name=collection_name, embedding_function=embeddings_model, auto_id=True)

    store = InMemoryStore()
    retriever = ParentDocumentRetriever(
        vectorstore=vector_store,
        docstore=store,
        child_splitter=child_splitter,
        parent_splitter=parent_splitter,
        k=10,
    )

    for i, doc in enumerate(tqdm(docs, desc="Añadiendo documentos al retriever")):
        try:
            if not doc.page_content.strip():
                logger.warning(f"Documento vacío encontrado en el índice {i}. Saltando...")
                continue
            
            retriever.add_documents([doc])
        except Exception as e:
            logger.error(f"Error al procesar el documento {i}: {str(e)}")
            logger.error(f"Contenido del documento: {doc.page_content[:100]}...")  # Muestra los primeros 100 caracteres
            raise

    return retriever, vector_store.as_retriever()
'''

In [471]:
len(documents)

6

In [472]:
parent_retriever, child_retriever = create_parent_retriever(documents, embedding_model)

In [473]:
#parent_retriever.get_relevant_documents(query=query)
parent_retriever.invoke(input=query)

[]

In [474]:
query

'Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?'

In [475]:
similar_chunks = retrieve_context(
        query, retriever=child_retriever,
    )
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content)

--------------------------------- chunk # 0 -------------------------------------
um Ihr Arbeitsverhältnis 12 Häufig gestellte Fragen 14 Services & Infos im Überblick Ein Überblick für den Start 16 Über die Uni Graz © Uni Graz/Konstantinov 5 Über die Universität Die Universität Graz, gegründet 1585, ist Österreichs zweitälteste Universität und eine der größten des Landes. Zahlreiche herausragende Wissenschafter:innen, unter ihnen sechs Nobelpreisträger, haben hier gelehrt und geforscht. Mit rund 30.000 Studierenden und 4.500 Mitarbeitenden trägt sie entscheidend zum pulsierenden Leben
--------------------------------- chunk # 1 -------------------------------------
am Institut für Öffentliches Recht und Politikwissenschaft und leitete bis September 2022 die Abteilung Finanzen, Personal und Recht der Universität für Weiterbildung Krems. © Uni Graz/WildundWunderbar © Uni Graz/WildundWunderbar Organigramm FAKULTÄTEN* UNIRAT Der Unirat besteht aus 9
--------------------------------- chunk 

In [478]:
similar_chunks = retrieve_context(
        query, retriever=parent_retriever,
    )
for i, chunks in enumerate(similar_chunks):
    print(f"--------------------------------- chunk # {i} -------------------------------------")
    print(chunks.page_content)

In [477]:
prompt_template = """
Du bist ein hilfreicher Assistent. Gib Antworten auf die Fragen der Benutzer auf der Grundlage des bereitgestellten Kontexts.

Kontext:
{Kontext}

Frage:
{Frage}


"""
prompt = ChatPromptTemplate.from_template(prompt_template)

chain = {"Frage": RunnablePassthrough(), "Kontext": parent_retriever} \
        | prompt \
        | llm \
        | StrOutputParser() \

result = chain.invoke(query)

# display(HTML(result))
print(result)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext>] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "input": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "output": "Wer ist der Rektor der Universität Graz und in welchem Jahr wurde er geboren?"
}
[chain/end] [chain:RunnableSequence > chain:RunnableParallel<Frage,Kontext>] s] Exiting Chain run with output:
{
  "Frage": "Wer ist der Rektor der Universität Gr